In [1]:
# python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
# source elbaff_iesta_venv/bin/activate

In [2]:
from textmining_utility.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np
import sys, os
import importlib

import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY

from collections import Counter




/home/elba_ro/repos/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elba_ro/repos/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
conservative_each_dataloader = IESTAData(ideology=prop.CONSERVATIVE_IDEOLOGY, methodology=METHODOLOGY.EACH)





In [4]:
conservative_each_dataloader.load()

textmining-utility INFO     File already created. Loading file...
textmining-utility INFO     File already created. Loading file...


In [5]:
conservative_each_dataloader.pivot_df

effect,effective,ineffective,okay,provocative
split,,,,
test,294,8164,1677,93
training,2044,57196,11460,616
validation,649,16364,3178,197


In [6]:
conservative_each_dataloader.pivot_binary_effect

binary_effect,effective,ineffective
split,,
test,294,9934
training,2044,69272
validation,649,19739


In [7]:
conservative_training_each_df  = conservative_each_dataloader.data_df[conservative_each_dataloader.data_df["split"] == "training"].copy()
len(conservative_training_each_df)

71316

In [8]:
conservative_training_each_df.columns.tolist()

['id',
 'debate_id',
 'p_name',
 'effects',
 'effect_count',
 'top_effect',
 'effect',
 'category',
 'round',
 'argument',
 'split',
 'binary_effect']

In [9]:
import dataclasses
class TransformerBasedFeaturePipeline(Pipeline):
    
    def __init__(self,
                 input= None,
                 load_default_pipe_configs = True,
                 extended_pipe_configs:dict = None,
                 save_output= False,
                 out_path = None,
                 argument_col:str = "argument"
                 ):
        super().__init__(input,
                 load_default_pipe_configs,
                 extended_pipe_configs,
                 save_output,
                 out_path)
        self.argument_col= argument_col
        
    def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", self.argument_col]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", self.argument_col: "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed

    def init_and_run(self):
        
        self.add_annotation_pipe(name = "senter",                  save_output= False,is_spacy=True, is_native=True)
        self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)

        # annotate the input
        #self.set_spacy_language_model("en_core_web_md")
        self.annotate()
        # save annotations when "save_output" is set to True
        self.save()

In [10]:
pipeline = TransformerBasedFeaturePipeline(save_output= True,
                 out_path = "../data/extracted_features/features_conservative_training_each.parquet")
pipeline.set_input(conservative_training_each_df)
#pipeline.set_input(test_df)
pipeline.init_and_run()
pipeline.out_df.head()

textmining-utility INFO     adding pipe with name EmotionPipeOrchestrator
textmining-utility INFO     adding pipe with name EmotionPipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.emotion
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.emotion
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code emotion_hartmann_component
textmining-utility INFO     adding pipe with code emotion_hartmann_component
textmining-utility INFO     adding pipe with name HedgePipeOrchestrator
textmining-utility INFO     adding pipe with name HedgePipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.hedge
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.hedge
textmining-utility INFO     orchestrator was initialized succe

In [ ]:
pipeline.spacy_language_model

In [ ]:
lib_transformers_df = pd.read_parquet("../data/extracted_features/features_liberal_training_each.parquet")

lib_transformers_df.head()
lib_transformers_df['hedge_dominant'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['emotion_hartmann_label'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['toxicity_dominant'].value_counts().to_frame().plot(kind="bar")

lib_transformers_df[["emotion_hartmann_anger",
                     "emotion_hartmann_disgust",
                    "emotion_hartmann_fear", 
                    "emotion_hartmann_joy",
                    "emotion_hartmann_neutral",
                    "emotion_hartmann_surprise",
                    "emotion_hartmann_sadness"]].plot.hist(bins=12, alpha=0.5)